<a href="https://colab.research.google.com/github/jptom/bengali_classification/blob/master/graphemes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!cp /content/drive/MyDrive/T0828/term\ project/dataset/graphemes.zip .
!cp /content/drive/MyDrive/T0828/term\ project/dataset/train_labels.csv .
!cp /content/drive/MyDrive/T0828/term\ project/dataset/val_labels.csv .

In [10]:
!unzip /content/graphemes.zip -d ./

Archive:  /content/graphemes.zip
replace ./graphemes/Train_32026.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ./graphemes/Train_84338.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ./graphemes/Train_85015.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ./graphemes/Train_172116.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ./graphemes/Train_58197.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [22]:
import pandas as pd
import tensorflow.math as tfm
import tensorflow.keras as keras
import tensorflow.keras.backend as kb
import tensorflow.keras.layers as kl
import tensorflow.keras.applications as ka
import tensorflow.keras.utils as ku
import tensorflow.keras.preprocessing.image as ki
import numpy as np
import os
img_dir = "graphemes"
exten = ".jpg"

In [28]:
id="image_id"
num_classes=(168, 11, 8)
classnames=("grapheme_root" ,"vowel_diacritic", "consonant_diacritic")
class GraphemesDataset(ku.Sequence):
    def __init__(self, data:pd.DataFrame, batch_size=64,):
        self.data = data.sample(frac=1).reset_index(drop=True)
        self.batch_size=batch_size
        
    def on_epoch_end(self):
        self.data = self.data.sample(frac=1).reset_index(drop=True)

    def __getitem__(self, index):
        batch_data = self.data[index*self.batch_size:(index+1)*self.batch_size]
        batch_x = self.load_imgs_to_array(batch_data)
        batch_c0, batch_c1, batch_c2 = self.make_labels(batch_data)
        return (batch_x, batch_c0, batch_c1, batch_c2), (batch_c0, batch_c1, batch_c2)

    def __len__(self):
        return int(np.floor(len(self.data) / self.batch_size))

    def load_imgs_to_array(self, df):
        batch_x = []
        for filename in df[id]:
            img = ki.load_img(os.path.join("./", img_dir, filename+exten))
            array = ki.img_to_array(img)/255
            batch_x.append(array)
        return np.array(batch_x)

    def make_labels(self, df:pd.DataFrame):
        c0 = ku.to_categorical(df[classnames[0]], num_classes=num_classes[0])
        c1 = ku.to_categorical(df[classnames[1]], num_classes=num_classes[1])
        c2 = ku.to_categorical(df[classnames[2]], num_classes=num_classes[2])
        return c0, c1, c2

In [29]:
train_df = pd.read_csv("/content/drive/MyDrive/T0828/term project/dataset/train_labels.csv")
val_df = pd.read_csv("/content/drive/MyDrive/T0828/term project/dataset/val_labels.csv")
trainset = GraphemesDataset(train_df)
valset = GraphemesDataset(val_df)

In [30]:
def GraphemesAccuracy(y_true, y_pred):
    print(y_true.shape, y_pred.shape)
    if y_true.shape[0] == None:
        print("f")
        return 1
    c0_tf = kb.argmax(y_true[:, num_classes[0]], axis=-1) == kb.argmax(y_pred[:, num_classes[0]], axis=-1)
    c1_tf = kb.argmax(y_true[:, num_classes[0]:num_classes[0]+num_classes[1]], axis=-1) == kb.argmax(y_pred[:, num_classes[0]:num_classes[0]+num_classes[1]], axis=-1)
    c2_tf = kb.argmax(y_true[:, num_classes[0]+num_classes[1]:], axis=-1) == kb.argmax(y_pred[:, num_classes[0]+num_classes[1]:], axis=-1)
    print(c0_tf.shape, c1_tf.shape, c2_tf.shape)
    overall = kb.all(kb.stack((c0_tf, c1_tf, c2_tf), axis=-1), axis=-1)
    return kb.mean(overall)

def GraphemesLoss(y_true, y_pred):
    print(len(y_treu), len(y_pred))
    return 1

In [25]:
class ArcFace(kl.Layer):
    def __init__(self, n_classes=10, s=30.0, m=0.50, regularizer=None, **kwargs):
        super(ArcFace, self).__init__(**kwargs)
        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.regularizer = keras.regularizers.get(regularizer)

    def build(self, input_shape):
        super(ArcFace, self).build(input_shape[0])
        self.W = self.add_weight(name='W',
                                shape=(input_shape[0][-1], self.n_classes),
                                initializer='glorot_uniform',
                                trainable=True,
                                regularizer=self.regularizer)

    def call(self, inputs):
        x, y = inputs
        c = kb.shape(x)[-1]
        # normalize feature
        x = kb.l2_normalize(x, axis=1)
        # normalize weights
        W = kb.l2_normalize(self.W, axis=0)
        # dot product
        logits = x @ W
        # add margin
        # clip logits to prevent zero division when backward
        theta = tfm.acos(kb.clip(logits, -1.0 + kb.epsilon(), 1.0 - kb.epsilon()))
        target_logits = tfm.cos(theta + self.m)
        # sin = tf.sqrt(1 - logits**2)
        # cos_m = tf.cos(logits)
        # sin_m = tf.sin(logits)
        # target_logits = logits * cos_m - sin * sin_m
        #
        logits = logits * (1 - y) + target_logits * y
        # feature re-scale
        logits *= self.s
        out = kb.softmax(logits)

        return out

    def compute_output_shape(self, input_shape):
        return (None, self.n_classes)

In [27]:
img_shape = (224, 224, 3)

model = ka.EfficientNetB0(classifier_activation=None, )

img_inputs = kl.Input(shape=img_shape)
c0_y = kl.Input(shape=(168))
c1_y = kl.Input(shape=(11))
c2_y = kl.Input(shape=(8))

c0_classifier = ArcFace(n_classes=168)
c1_classifier = ArcFace(n_classes=11)
c2_classifier = ArcFace(n_classes=8)

x = model(img_inputs)
c0_outputs = c0_classifier((x, c0_y))
c1_outputs = c1_classifier((x, c1_y))
c2_outputs = c2_classifier((x, c2_y))


MyModel = keras.Model(inputs=[img_inputs, c0_y, c1_y, c2_y], outputs=[c0_outputs, c1_outputs, c2_outputs])
MyModel.summary()
#print(model.compute_output_shape(input_shape=(64, 128, 128, 3)))
model.compile(optimizer="sgd", loss=GraphemesLoss)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_53 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
efficientnetb0 (Functional)     (None, 1000)         5330571     input_53[0][0]                   
__________________________________________________________________________________________________
input_54 (InputLayer)           [(None, 168)]        0                                            
__________________________________________________________________________________________________
input_55 (InputLayer)           [(None, 11)]         0                                            
______________________________________________________________________________________________

In [118]:
model.fit(x=trainset, epochs=1, callbacks=None, validation_data=valset, steps_per_epoch=10)

(None, None) Tensor("efficientnetb0/predictions/Sigmoid:0", shape=(None, 187), dtype=float32)
(None, None) Tensor("efficientnetb0/predictions/Sigmoid:0", shape=(None, 187), dtype=float32)
10/10 [==============================] - 36s 3s/step - loss: 0.7282 - mse: 1.0000 - val_loss: 0.6927 - val_mse: 1.0000


In [33]:
kb.argmax(np.array([[0, 0, 1], [0, 1, 0], [1, 0, 0]]))

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([2, 1, 0])>

In [45]:
kb.mean(kb.all(np.array([[1, 1, 1], [0, 1, 0], [1, 0, 0]]), axis=1))

<tf.Tensor: shape=(), dtype=float32, numpy=0.33333334>

In [75]:
a = np.array([1, 2, 3])
b = np.array([3, 2, 1])
a == b
kb.equal(a, b)

<tf.Tensor: shape=(3,), dtype=bool, numpy=array([False,  True, False])>

In [51]:
import tensorflow as tf
a = tf.constant([1, 2])
b = tf.constant([10, 20])
kb.stack((a, b), axis=-1)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 1, 10],
       [ 2, 20]], dtype=int32)>

In [ ]:
kb.equal